In [ ]:
# Section 1: Import libraries and preprocess data

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Flatten, Dense, Reshape
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import numpy as np

# Load Fashion MNIST from Keras (no download required)
(x_train, _), (x_test, _) = tf.keras.datasets.fashion_mnist.load_data()

# Normalize the data to [0, 1]
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Reshape the data to (28, 28, 1) for grayscale image
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")



29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


In [ ]:
# Section 2: Define the autoencoder architecture

# Define the input layer
input_img = Input(shape=(28, 28, 1))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Flatten()(x)
latent_vector = Dense(64, activation='relu')(x)  # Latent space with 64 units

# Decoder
x = Dense(28 * 28 * 64, activation='relu')(latent_vector)
x = Reshape((28, 28, 64))(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
output_img = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

# Define the autoencoder model
autoencoder = Model(inputs=input_img, outputs=output_img)

# Compile the model with Mean Squared Error loss
autoencoder.compile(optimizer='adam', loss='mse')

# Display the model summary
autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 28, 28, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 28, 28, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       3,211,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50176)               │       3,261,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 28, 28, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 28, 28, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 28, 28, 1)           │             289 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,547,265 (24.98 MB)

 Trainable params: 6,547,265 (24.98 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Section 3: Train the autoencoder

# Train the autoencoder
history = autoencoder.fit(
    x_train, x_train,
    epochs=10,
    batch_size=32,
    validation_data=(x_test, x_test)
)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 832s 443ms/step - loss: 0.0291 - val_loss: 0.0134
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 856s 439ms/step - loss: 0.0130 - val_loss: 0.0123
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 828s 441ms/step - loss: 0.0120 - val_loss: 0.0121
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 849s 435ms/step - loss: 0.0115 - val_loss: 0.0116
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 849s 428ms/step - loss: 0.0112 - val_loss: 0.0114
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 870s 432ms/step - loss: 0.0109 - val_loss: 0.0109
Epoch 7/10
 643/1875 ━━━━━━━━━━━━━━━━━━━━ 8:26 411ms/step - loss: 0.0103

In [ ]:
# Section 4: Visualize the results (original vs reconstructed images)

def display_reconstructed_images(model, test_images, num_images=10):
    decoded_imgs = model.predict(test_images[:num_images])
    plt.figure(figsize=(20, 4))
    for i in range(num_images):
        # Original images
        ax = plt.subplot(2, num_images, i + 1)
        plt.imshow(test_images[i].reshape(28, 28), cmap='gray')
        plt.axis("off")

        # Reconstructed images
        ax = plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
        plt.axis("off")

    plt.show()

# Display a few original and reconstructed images
display_reconstructed_images(autoencoder, x_test)


In [ ]:
# Section 5: Extract Latent Representations and Use t-SNE for Visualization

# Extract the encoder part of the autoencoder
encoder = Model(inputs=input_img, outputs=latent_vector)

# Get latent representations for the test set
latent_representations = encoder.predict(x_test)

# Use t-SNE for dimensionality reduction to 2D
latent_2d = TSNE(n_components=2, random_state=42).fit_transform(latent_representations)

# Visualize the Latent Space with t-SNE
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=latent_2d[:, 0], y=latent_2d[:, 1],
    hue=_[:len(x_test)], palette='tab10', s=10, legend='full'
)
plt.title("2D Visualization of Latent Space (Colored by Labels)")
plt.xlabel("Latent Dimension 1")
plt.ylabel("Latent Dimension 2")
plt.legend(title="Clothing Categories")
plt.show()


In [ ]:
# Section 6: Interpolation Between Latent Vectors

# Function to interpolate between two latent vectors
def interpolate_latent_space(start_latent, end_latent, num_steps=10):
    return np.linspace(start_latent, end_latent, num_steps)

# Select two random latent vectors from the latent space
start_latent = latent_representations[0]  # Example: first test image
end_latent = latent_representations[1]    # Example: second test image

# Interpolate between the two latent vectors
interpolated_latents = interpolate_latent_space(start_latent, end_latent, num_steps=10)

# Generate the images from the interpolated latent vectors
interpolated_images = autoencoder.predict(interpolated_latents)

# Plot the interpolated images
plt.figure(figsize=(20, 4))
for i, img in enumerate(interpolated_images):
    plt.subplot(1, 10, i + 1)
    plt.imshow(img.reshape(28, 28), cmap='gray')
    plt.axis("off")
plt.show()
